# ⚡ Aula 3 – GPUs em Python e Aplicações em Engenharia

## Computação de Alto Desempenho em Python para Engenharia Civil

**Objetivos desta aula:**
- Introduzir paralelismo massivo com GPU
- Usar CuPy (NumPy para GPU) e Numba CUDA
- Aplicar HPC a simulações reais (difusão de calor)
- Comparar performance CPU vs GPU

---

### 🎯 CPU vs GPU: Filosofias Diferentes

**CPU (Central Processing Unit):**
- Poucos núcleos complexos (4-64 cores)
- Otimizada para latência (velocidade individual)
- Hierarquia de cache complexa
- Ideal para código sequencial e ramificado

**GPU (Graphics Processing Unit):**
- Milhares de núcleos simples (1000+ cores)
- Otimizada para throughput (trabalho total)
- Arquitetura SIMD (Single Instruction, Multiple Data)
- Ideal para paralelismo massivo e dados regulares

### 🏗️ Aplicações GPU em Engenharia Civil
- **Simulações CFD** (Computational Fluid Dynamics)
- **Análise modal** de estruturas complexas
- **Processamento de imagens** (inspeção, monitoramento)
- **Machine Learning** para predição de falhas

In [ ]:
# Import Required Libraries
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import multiprocessing as mp

# Tentar importar bibliotecas GPU
try:
    import cupy as cp
    CUPY_AVAILABLE = True
    print(f"✅ CuPy {cp.__version__} disponível")
    print(f"   GPU: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
    print(f"   Memória: {cp.cuda.runtime.memGetInfo()[1] // (1024**3)} GB")
except ImportError:
    CUPY_AVAILABLE = False
    print("⚠️  CuPy não disponível - instale com: pip install cupy-cuda11x (ou cupy-cuda12x)")

try:
    from numba import cuda
    import numba
    NUMBA_CUDA_AVAILABLE = True
    print(f"✅ Numba CUDA {numba.__version__} disponível")
except ImportError:
    NUMBA_CUDA_AVAILABLE = False
    print("⚠️  Numba CUDA não disponível")

# Configure matplotlib
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (12, 8)

print(f"n🖥️  Sistema: {mp.cpu_count()} núcleos CPU")
print(f"📊 NumPy: {np.__version__}")

if not CUPY_AVAILABLE and not NUMBA_CUDA_AVAILABLE:
    print("n⚠️  Nota: Esta aula requer GPU NVIDIA e CUDA para funcionalidade completa")
    print("   Mas ainda podemos aprender os conceitos e ver simulações CPU!")

## 1. Exemplo 10: Soma de Vetores com CuPy

CuPy oferece uma interface NumPy para GPUs - mudança mínima de código!

In [ ]:
if CUPY_AVAILABLE:
    print("🚀 Exemplo 10: Soma de Vetores com CuPy")
    print("=" * 40)
    
    # Teste com diferentes tamanhos de vetores
    sizes = [1_000_000, 10_000_000, 100_000_000]
    
    for N in sizes:
        print(f"n📊 Tamanho: {N:,} elementos")
        
        # Criar vetores no CPU
        print("  Criando dados no CPU...")
        a_cpu = np.random.randn(N).astype(np.float32)
        b_cpu = np.random.randn(N).astype(np.float32)
        
        # NumPy (CPU)
        start = time.perf_counter()
        c_cpu = a_cpu + b_cpu
        time_cpu = time.perf_counter() - start
        print(f"  NumPy (CPU):     {time_cpu:.4f}s")
        
        # Transferir para GPU
        start_transfer = time.perf_counter()
        a_gpu = cp.asarray(a_cpu)
        b_gpu = cp.asarray(b_cpu)
        time_transfer_to = time.perf_counter() - start_transfer
        
        # CuPy (GPU) - primeira execução
        start = time.perf_counter()
        c_gpu = a_gpu + b_gpu
        cp.cuda.Stream.null.synchronize()  # Aguardar conclusão
        time_gpu_first = time.perf_counter() - start
        
        # CuPy (GPU) - segunda execução (kernels já compilados)
        start = time.perf_counter()
        c_gpu = a_gpu + b_gpu
        cp.cuda.Stream.null.synchronize()
        time_gpu = time.perf_counter() - start
        
        # Transferir resultado de volta
        start_transfer = time.perf_counter()
        c_gpu_cpu = cp.asnumpy(c_gpu)
        time_transfer_back = time.perf_counter() - start_transfer
        
        # Verificar correção
        are_equal = np.allclose(c_cpu, c_gpu_cpu, rtol=1e-5)
        
        # Speedup considerando apenas computação
        speedup_compute = time_cpu / time_gpu
        
        # Speedup total (incluindo transferências)
        total_gpu_time = time_transfer_to + time_gpu + time_transfer_back
        speedup_total = time_cpu / total_gpu_time
        
        print(f"  CuPy (GPU):      {time_gpu:.4f}s")
        print(f"  Transferência:   {time_transfer_to + time_transfer_back:.4f}s")
        print(f"  Total GPU:       {total_gpu_time:.4f}s")
        print(f"  Speedup (comp):  {speedup_compute:.2f}x")
        print(f"  Speedup (total): {speedup_total:.2f}x")
        print(f"  Precisão:        {'✓' if are_equal else '✗'}")
    
    print("n💡 Observações importantes:")
    print("• Transferência CPU↔GPU é custosa")
    print("• Speedup real depende do tamanho do problema")
    print("• GPU é mais eficiente para problemas grandes")
    print("• CuPy = np → cp (mudança mínima de código)")

else:
    print("⚠️  CuPy não disponível. Exemplo conceitual:")
    print("""
    # NumPy (CPU)
    import numpy as np
    a = np.arange(10_000_000)
    b = np.arange(10_000_000)
    c = a + b  # Executa no CPU
    
    # CuPy (GPU) - mudança mínima!
    import cupy as cp
    a = cp.arange(10_000_000)  # Criado diretamente na GPU
    b = cp.arange(10_000_000)
    c = a + b  # Executa na GPU automaticamente!
    """)

## 2. Exemplo 11: Multiplicação de Matrizes Massivas

Para matrizes grandes, GPUs mostram seu verdadeiro poder!

In [ ]:
if CUPY_AVAILABLE:
    print("🧮 Exemplo 11: Multiplicação de Matrizes Massivas")
    print("=" * 50)
    
    # Testar diferentes tamanhos de matrizes
    matrix_sizes = [1024, 2048, 4096]
    
    for size in matrix_sizes:
        print(f"n📊 Matrizes {size}x{size} (elementos: {size**2:,})")
        
        # Criar matrizes aleatórias
        print("  Gerando matrizes aleatórias...")
        np.random.seed(42)
        A_cpu = np.random.randn(size, size).astype(np.float32)
        B_cpu = np.random.randn(size, size).astype(np.float32)
        
        # NumPy (CPU) com BLAS otimizado
        print("  Executando multiplicação no CPU...")
        start = time.perf_counter()
        C_cpu = np.dot(A_cpu, B_cpu)
        time_cpu = time.perf_counter() - start
        print(f"  NumPy (CPU):     {time_cpu:.4f}s")
        
        # Transferir para GPU
        start = time.perf_counter()
        A_gpu = cp.asarray(A_cpu)
        B_gpu = cp.asarray(B_cpu)
        time_transfer_to = time.perf_counter() - start
        print(f"  Transferência→GPU: {time_transfer_to:.4f}s")
        
        # CuPy (GPU)
        print("  Executando multiplicação na GPU...")
        start = time.perf_counter()
        C_gpu = cp.dot(A_gpu, B_gpu)
        cp.cuda.Stream.null.synchronize()
        time_gpu = time.perf_counter() - start
        print(f"  CuPy (GPU):      {time_gpu:.4f}s")
        
        # Transferir resultado de volta
        start = time.perf_counter()
        C_gpu_cpu = cp.asnumpy(C_gpu)
        time_transfer_back = time.perf_counter() - start
        print(f"  Transferência←GPU: {time_transfer_back:.4f}s")
        
        # Verificar precisão
        max_error = np.max(np.abs(C_cpu - C_gpu_cpu))
        are_close = np.allclose(C_cpu, C_gpu_cpu, rtol=1e-4)
        
        # Calcular speedups
        speedup_compute = time_cpu / time_gpu
        total_gpu_time = time_transfer_to + time_gpu + time_transfer_back
        speedup_total = time_cpu / total_gpu_time
        
        # Calcular FLOPS (operações de ponto flutuante por segundo)
        flops = 2 * size**3  # Multiplicação de matrizes: 2*n³ operações
        gflops_cpu = flops / (time_cpu * 1e9)
        gflops_gpu = flops / (time_gpu * 1e9)
        
        print(f"n  📈 Performance:")
        print(f"    CPU GFLOPS:      {gflops_cpu:.2f}")
        print(f"    GPU GFLOPS:      {gflops_gpu:.2f}")
        print(f"    Speedup (comp):  {speedup_compute:.2f}x")
        print(f"    Speedup (total): {speedup_total:.2f}x")
        print(f"    Erro máximo:     {max_error:.2e}")
        print(f"    Precisão:        {'✓' if are_close else '✗'}")
        
        # Limpeza de memória GPU
        del A_gpu, B_gpu, C_gpu
        cp.get_default_memory_pool().free_all_blocks()
    
    print("n💡 Observações:")
    print("• GPU acelera significativamente operações matriciais grandes")
    print("• GFLOPS (Giga FLOPS) mede eficiência computacional")
    print("• Speedup melhora com tamanho do problema")
    print("• Essencial considerar overhead de transferência")

else:
    print("⚠️  Demonstração conceitual de multiplicação de matrizes GPU:")
    
    # Simular performance típica
    sizes = [1024, 2048, 4096]
    print("Speedups típicos para multiplicação de matrizes:")
    print("Tamanho    CPU (s)    GPU (s)    Speedup")
    print("-" * 40)
    
    for size in sizes:
        # Estimativas baseadas em hardware típico
        flops = 2 * size**3
        time_cpu_est = flops / (100e9)  # ~100 GFLOPS CPU
        time_gpu_est = flops / (2000e9)  # ~2000 GFLOPS GPU
        speedup_est = time_cpu_est / time_gpu_est
        
        print(f"{size:>6}x{size:<6} {time_cpu_est:.3f}     {time_gpu_est:.4f}    {speedup_est:.1f}x")

## 3. Introdução ao Numba CUDA

Numba CUDA permite escrever kernels customizados para máximo controle sobre a GPU.

In [ ]:
if NUMBA_CUDA_AVAILABLE:
    print("🔧 Introdução ao Numba CUDA")
    print("=" * 30)
    
    @cuda.jit
    def add_kernel(a, b, c):
        # """Kernel CUDA para soma elemento por elemento"""
        # Obter índice do thread atual
        i = cuda.grid(1)
        
        # Verificar bounds
        if i < a.size:
            c[i] = a[i] + b[i]
    
    @cuda.jit 
    def square_kernel(arr, result):
        # """Kernel CUDA para elevar ao quadrado"""
        i = cuda.grid(1)
        
        if i < arr.size:
            result[i] = arr[i] * arr[i]
    
    # Exemplo 12: Monte Carlo π com CUDA
    @cuda.jit
    def monte_carlo_pi_kernel(rng_states, n_per_thread, results):
        # """Kernel para estimativa de π por Monte Carlo"""
        thread_id = cuda.grid(1)
        
        if thread_id >= rng_states.size:
            return
            
        # Cada thread conta pontos dentro do círculo
        count = 0
        for i in range(n_per_thread):
            x = cuda.random.xoroshiro128p_uniform_float32(rng_states, thread_id)
            y = cuda.random.xoroshiro128p_uniform_float32(rng_states, thread_id)
            
            if x*x + y*y <= 1.0:
                count += 1
        
        results[thread_id] = count
    
    def monte_carlo_pi_cuda(n_samples, n_threads=256):
        # """Estimativa de π usando CUDA"""
        n_blocks = (n_samples + n_threads - 1) // n_threads
        samples_per_thread = n_samples // (n_blocks * n_threads)
        
        # Alocar arrays na GPU
        rng_states = cuda.random.create_xoroshiro128p_states(n_blocks * n_threads, seed=42)
        results = cuda.device_array(n_blocks * n_threads, dtype=np.int32)
        
        # Executar kernel
        start = time.perf_counter()
        monte_carlo_pi_kernel[n_blocks, n_threads](rng_states, samples_per_thread, results)
        cuda.synchronize()
        time_gpu = time.perf_counter() - start
        
        # Transferir resultados e somar
        results_host = results.copy_to_host()
        total_inside = np.sum(results_host)
        total_samples = samples_per_thread * n_blocks * n_threads
        
        pi_estimate = 4.0 * total_inside / total_samples
        return pi_estimate, time_gpu
    
    print("n🎲 Exemplo 12: Monte Carlo π com Numba CUDA")
    print("-" * 45)
    
    # Teste com diferentes números de amostras
    sample_counts = [1_000_000, 10_000_000, 100_000_000]
    
    for n_samples in sample_counts:
        print(f"nAmostras: {n_samples:,}")
        
        # Versão CPU para comparação
        start = time.perf_counter()
        np.random.seed(42)
        x_cpu = np.random.uniform(-1, 1, n_samples)
        y_cpu = np.random.uniform(-1, 1, n_samples)
        inside_cpu = np.sum((x_cpu**2 + y_cpu**2) <= 1)
        pi_cpu = 4 * inside_cpu / n_samples
        time_cpu = time.perf_counter() - start
        
        # Versão GPU
        pi_gpu, time_gpu = monte_carlo_pi_cuda(n_samples)
        
        speedup = time_cpu / time_gpu
        error_cpu = abs(pi_cpu - np.pi)
        error_gpu = abs(pi_gpu - np.pi)
        
        print(f"  CPU: π ≈ {pi_cpu:.6f}, erro = {error_cpu:.6f}, tempo = {time_cpu:.4f}s")
        print(f"  GPU: π ≈ {pi_gpu:.6f}, erro = {error_gpu:.6f}, tempo = {time_gpu:.4f}s")
        print(f"  Speedup: {speedup:.2f}x")
    
    print("n💡 Conceitos CUDA importantes:")
    print("• Grid: coleção de blocos")
    print("• Block: coleção de threads")
    print("• Thread: unidade de execução")
    print("• cuda.grid(1): obtém índice global do thread")
    print("• cuda.synchronize(): aguarda conclusão")

else:
    print("⚠️  Numba CUDA não disponível. Conceitos fundamentais:")
    print("""
    Hierarquia CUDA:
    
    Grid (toda a GPU)
    ├── Block 0 (grupo de threads)
    │   ├── Thread 0
    │   ├── Thread 1
    │   └── Thread N
    ├── Block 1
    └── Block M
    
    Kernel CUDA exemplo:
    
    @cuda.jit
    def meu_kernel(input_array, output_array):
        # Obter índice único para este thread
        i = cuda.grid(1)
        
        # Verificar bounds
        if i < input_array.size:
            output_array[i] = input_array[i] * 2
    
    # Lançar kernel
    threads_per_block = 256
    blocks_per_grid = (array.size + threads_per_block - 1) // threads_per_block
    meu_kernel[blocks_per_grid, threads_per_block](input_arr, output_arr)
    """)

## 4. Exemplo 13: Simulação de Difusão de Calor 2D

Vamos implementar uma simulação realística de engenharia: difusão de calor em uma placa metálica.

In [ ]:
def heat_equation_cpu(T, alpha, dx, dy, dt, steps):
    """
    Simula difusão de calor 2D no CPU
    Equação: ∂T/∂t = α(∂²T/∂x² + ∂²T/∂y²)
    """
    ny, nx = T.shape
    T_new = T.copy()
    
    start = time.perf_counter()
    
    for step in range(steps):
        # Atualizar pontos internos usando diferenças finitas
        for i in range(1, ny-1):
            for j in range(1, nx-1):
                T_new[i, j] = T[i, j] + alpha * dt * (
                    (T[i+1, j] - 2*T[i, j] + T[i-1, j]) / dx**2 +
                    (T[i, j+1] - 2*T[i, j] + T[i, j-1]) / dy**2
                )
        
        # Trocar arrays
        T, T_new = T_new, T
    
    exec_time = time.perf_counter() - start
    return T, exec_time

if CUPY_AVAILABLE:
    def heat_equation_gpu(T_gpu, alpha, dx, dy, dt, steps):
        """Simula difusão de calor 2D na GPU usando CuPy"""
        ny, nx = T_gpu.shape
        T_new_gpu = cp.copy(T_gpu)
        
        start = time.perf_counter()
        
        for step in range(steps):
            # Atualizar usando slicing vetorizado
            T_new_gpu[1:-1, 1:-1] = T_gpu[1:-1, 1:-1] + alpha * dt * (
                (T_gpu[2:, 1:-1] - 2*T_gpu[1:-1, 1:-1] + T_gpu[:-2, 1:-1]) / dx**2 +
                (T_gpu[1:-1, 2:] - 2*T_gpu[1:-1, 1:-1] + T_gpu[1:-1, :-2]) / dy**2
            )
            
            # Trocar arrays
            T_gpu, T_new_gpu = T_new_gpu, T_gpu
            
        cp.cuda.Stream.null.synchronize()
        exec_time = time.perf_counter() - start
        return T_gpu, exec_time

if NUMBA_CUDA_AVAILABLE:
    @cuda.jit
    def heat_equation_kernel(T_old, T_new, alpha, dx, dy, dt):
        """Kernel CUDA para um passo da equação de calor"""
        i, j = cuda.grid(2)  # Obter coordenadas 2D
        
        ny, nx = T_old.shape
        
        # Verificar bounds (evitar bordas)
        if 1 <= i < ny-1 and 1 <= j < nx-1:
            T_new[i, j] = T_old[i, j] + alpha * dt * (
                (T_old[i+1, j] - 2*T_old[i, j] + T_old[i-1, j]) / (dx*dx) +
                (T_old[i, j+1] - 2*T_old[i, j] + T_old[i, j-1]) / (dy*dy)
            )
    
    def heat_equation_cuda(T_host, alpha, dx, dy, dt, steps):
        """Simula difusão de calor 2D usando kernels CUDA"""
        ny, nx = T_host.shape
        
        # Alocar na GPU
        T_gpu = cuda.to_device(T_host)
        T_new_gpu = cuda.device_array_like(T_gpu)
        
        # Configurar grid de threads
        threads_per_block = (16, 16)
        blocks_per_grid_x = (nx + threads_per_block[0] - 1) // threads_per_block[0]
        blocks_per_grid_y = (ny + threads_per_block[1] - 1) // threads_per_block[1]
        blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)
        
        start = time.perf_counter()
        
        for step in range(steps):
            heat_equation_kernel[blocks_per_grid, threads_per_block](
                T_gpu, T_new_gpu, alpha, dx, dy, dt
            )
            
            # Trocar buffers
            T_gpu, T_new_gpu = T_new_gpu, T_gpu
        
        cuda.synchronize()
        exec_time = time.perf_counter() - start
        
        # Transferir resultado de volta
        result = T_gpu.copy_to_host()
        return result, exec_time

print("🌡️  Exemplo 13: Simulação de Difusão de Calor 2D")
print("=" * 50)

# Parâmetros físicos
Lx, Ly = 1.0, 1.0  # Dimensões da placa (metros)
nx, ny = 256, 256  # Pontos da malha
dx = Lx / (nx - 1)
dy = Ly / (ny - 1)
alpha = 1e-4  # Difusividade térmica (m²/s)
dt = 0.25 * min(dx, dy)**2 / (4 * alpha)  # Passo de tempo estável
steps = 500  # Número de passos de tempo

print(f"Parâmetros da simulação:")
print(f"  Malha: {nx}x{ny} = {nx*ny:,} pontos")
print(f"  Passos de tempo: {steps}")
print(f"  dt = {dt:.2e}s (estabilidade: {dt*alpha/(dx**2):.3f} < 0.25)")

# Condições iniciais e de contorno
T_initial = np.zeros((ny, nx), dtype=np.float32)

# Fonte de calor no centro
center_x, center_y = nx//2, ny//2
radius = min(nx, ny) // 8
for i in range(ny):
    for j in range(nx):
        if (i - center_y)**2 + (j - center_x)**2 <= radius**2:
            T_initial[i, j] = 100.0  # 100°C

# Bordas mantidas a 0°C (condição de Dirichlet)
T_initial[0, :] = T_initial[-1, :] = 0.0
T_initial[:, 0] = T_initial[:, -1] = 0.0

print(f"nCondições iniciais: fonte de calor central a 100°C")

# Simular no CPU
print("n⏱️  Executando simulação no CPU...")
T_cpu, time_cpu = heat_equation_cpu(T_initial.copy(), alpha, dx, dy, dt, steps)
print(f"CPU: {time_cpu:.3f}s")

# Simular na GPU (se disponível)
if CUPY_AVAILABLE:
    print("n⏱️  Executando simulação na GPU (CuPy)...")
    T_gpu_cupy, time_gpu_cupy = heat_equation_gpu(cp.asarray(T_initial), alpha, dx, dy, dt, steps)
    T_gpu_cupy_host = cp.asnumpy(T_gpu_cupy)
    
    speedup_cupy = time_cpu / time_gpu_cupy
    max_diff_cupy = np.max(np.abs(T_cpu - T_gpu_cupy_host))
    
    print(f"GPU (CuPy): {time_gpu_cupy:.3f}s, speedup: {speedup_cupy:.2f}x")
    print(f"Diferença máxima CPU vs GPU: {max_diff_cupy:.2e}°C")

if NUMBA_CUDA_AVAILABLE:
    print("n⏱️  Executando simulação na GPU (Numba CUDA)...")
    T_gpu_cuda, time_gpu_cuda = heat_equation_cuda(T_initial.copy(), alpha, dx, dy, dt, steps)
    
    speedup_cuda = time_cpu / time_gpu_cuda
    max_diff_cuda = np.max(np.abs(T_cpu - T_gpu_cuda))
    
    print(f"GPU (CUDA): {time_gpu_cuda:.3f}s, speedup: {speedup_cuda:.2f}x")
    print(f"Diferença máxima CPU vs CUDA: {max_diff_cuda:.2e}°C")

# Visualização dos resultados
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Estado inicial
im1 = axes[0].imshow(T_initial, cmap='hot', interpolation='bilinear')
axes[0].set_title('Estado Inicialn(t = 0)')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
plt.colorbar(im1, ax=axes[0], label='Temperatura (°C)')

# Estado final (CPU)
im2 = axes[1].imshow(T_cpu, cmap='hot', interpolation='bilinear')
axes[1].set_title(f'Estado Final (CPU)n(t = {steps*dt:.3f}s)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
plt.colorbar(im2, ax=axes[1], label='Temperatura (°C)')

# Comparação GPU vs CPU (se disponível)
if CUPY_AVAILABLE:
    diff = T_gpu_cupy_host - T_cpu
    im3 = axes[2].imshow(diff, cmap='RdBu_r', interpolation='bilinear')
    axes[2].set_title('Diferença GPU - CPUn(CuPy)')
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('y')
    plt.colorbar(im3, ax=axes[2], label='Diferença (°C)')
else:
    # Perfil de temperatura no centro
    center_line = T_cpu[ny//2, :]
    axes[2].plot(np.linspace(0, Lx, nx), center_line, 'r-', linewidth=2)
    axes[2].set_title('Perfil de Temperaturan(linha central)')
    axes[2].set_xlabel('Posição x (m)')
    axes[2].set_ylabel('Temperatura (°C)')
    axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("n💡 Aplicações em Engenharia Civil:")
print("• Análise térmica de estruturas de concreto")
print("• Simulação de incêndios em edificações")
print("• Comportamento térmico de pavimentos")
print("• Isolamento térmico de edifícios")
print("• Pontes térmicas em estruturas")

## 5. Integração CPU + GPU e Ferramentas Avançadas

Vamos ver como combinar CPU e GPU eficientemente e explorar ferramentas do ecossistema.

In [ ]:
print("🔗 Integração CPU + GPU e Ecossistema HPC")
print("=" * 45)

print("n📚 Ferramentas Avançadas do Ecossistema:")
print("n1. 🚀 **Dask + CuPy**: Computação distribuída em múltiplas GPUs")
print("   • Escalabilidade para clusters")
print("   • DataFrame distribuídos")
print("   • Exemplo: `import dask.array as da; da.from_array(cupy_array)`")

print("n2. 🧠 **PyTorch/TensorFlow**: Machine Learning em larga escala")
print("   • Redes neurais para previsão estrutural")
print("   • Processamento de imagens de inspeção")
print("   • Detecção de anomalias em estruturas")

print("n3. 🐳 **Singularity/Docker**: Containers para HPC")
print("   • Reprodutibilidade em clusters")
print("   • Isolamento de dependências")
print("   • Portabilidade entre sistemas")

print("n4. 📊 **Rapids**: Aceleração completa do pipeline de dados")
print("   • cuDF (pandas para GPU)")
print("   • cuML (scikit-learn para GPU)")
print("   • cuGraph (NetworkX para GPU)")

# Demonstração de pipeline híbrido CPU+GPU
def hybrid_data_processing_example():
    
    print("n💡 Exemplo de Pipeline Híbrido CPU+GPU:")
    print("n1. **Pré-processamento no CPU**:")
    print("   • Leitura de arquivos")
    print("   • Parsing de dados")
    print("   • Validação e limpeza")
    
    print("n2. **Computação intensiva na GPU**:")
    print("   • Simulações numéricas")
    print("   • Operações matriciais massivas")
    print("   • Processamento paralelo")
    
    print("n3. **Pós-processamento no CPU**:")
    print("   • Análise estatística")
    print("   • Geração de relatórios")
    print("   • Visualização")
    
    if CUPY_AVAILABLE:
        print("n🔧 Implementação prática:")
        
        # Simular dados de entrada (CPU)
        print("   Gerando dados no CPU...")
        data_cpu = np.random.randn(1000, 1000).astype(np.float32)
        
        # Transferir para GPU para processamento intensivo
        print("   Transferindo para GPU...")
        data_gpu = cp.asarray(data_cpu)
        
        # Operações intensivas na GPU
        print("   Processando na GPU...")
        start = time.perf_counter()
        
        # Simulação: múltiplas operações matriciais
        result_gpu = cp.dot(data_gpu, data_gpu.T)
        result_gpu = cp.linalg.eigvals(result_gpu)
        result_gpu = cp.sort(result_gpu)
        
        cp.cuda.Stream.null.synchronize()
        gpu_time = time.perf_counter() - start
        
        # Transferir de volta para análise no CPU
        print("   Transferindo resultado de volta...")
        result_cpu = cp.asnumpy(result_gpu)
        
        # Análise final no CPU
        print("   Analisando no CPU...")
        mean_eigenval = np.mean(result_cpu.real)
        std_eigenval = np.std(result_cpu.real)
        
        print(f"n📊 Resultados:")
        print(f"   Tempo GPU: {gpu_time:.4f}s")
        print(f"   Autovalor médio: {mean_eigenval:.6f}")
        print(f"   Desvio padrão: {std_eigenval:.6f}")
    
    else:
        print("n⚠️  CuPy não disponível para demonstração prática")

hybrid_data_processing_example()

print("n🎯 Boas Práticas para CPU+GPU:")
print("n• **Minimize transferências**: Mantenha dados na GPU o máximo possível")
print("• **Sobreposição**: Use streams CUDA para computação e transferência simultâneas")
print("• **Pinned memory**: Acelera transferências CPU↔GPU")
print("• **Batch processing**: Processe múltiplos itens juntos")
print("• **Profiling**: Use nvprof, Nsight para identificar gargalos")

print("n📈 Quando usar GPU vs CPU:")
print("n**Use GPU quando:**")
print("• Alto paralelismo (milhares de operações simultâneas)")
print("• Operações matemáticas intensivas")
print("• Dados regulares e homogêneos")
print("• Pouca ramificação condicional")

print("n**Use CPU quando:**")
print("• Código sequencial complexo")
print("• Muitas ramificações condicionais")
print("• I/O intensivo")
print("• Processamento de strings/texto")

print("n🔮 Futuro da Computação Heterogênea:")
print("• **SYCL/DPC++**: Programação unificada CPU/GPU/FPGA")
print("• **Intel oneAPI**: Toolchain multi-arquitetura")
print("• **AMD HIP**: Portabilidade CUDA→ROCm")
print("• **WebGPU**: Computação GPU no navegador")

# Exemplo de medição de throughput
if CUPY_AVAILABLE:
    print("n📊 Benchmark Final: Throughput de Dados")
    print("-" * 40)
    
    data_sizes = [1, 10, 100]  # MB
    
    for size_mb in data_sizes:
        n_elements = (size_mb * 1024 * 1024) // 4  # 4 bytes por float32
        
        # CPU
        data_cpu = np.random.randn(n_elements).astype(np.float32)
        start = time.perf_counter()
        result_cpu = np.sum(data_cpu**2)
        time_cpu = time.perf_counter() - start
        throughput_cpu = size_mb / time_cpu
        
        # GPU
        data_gpu = cp.asarray(data_cpu)
        start = time.perf_counter()
        result_gpu = cp.sum(data_gpu**2)
        cp.cuda.Stream.null.synchronize()
        time_gpu = time.perf_counter() - start
        throughput_gpu = size_mb / time_gpu
        
        print(f"{size_mb:3d} MB: CPU {throughput_cpu:6.1f} MB/s, GPU {throughput_gpu:6.1f} MB/s")